# CA1 - Used Car Prices

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import seaborn as sn

from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, validation_curve
from sklearn.neighbors import  KNeighborsClassifier
from sklearn.linear_model import LogisticRegression

data = pd.read_csv('used_cars_UK.csv')
data.head(3)

,Unnamed: 0,title,Price,Mileage(miles),Registration_Year,Previous Owners,Fuel type,Body type,Engine,Gearbox,Doors,Seats,Emission Class,Service history
0,0,SKODA Fabia,6900,70189,2016,3.0,Diesel,Hatchback,1.4L,Manual,5.0,5.0,Euro 6,NaN
1,1,Vauxhall Corsa,1495,88585,2008,4.0,Petrol,Hatchback,1.2L,Manual,3.0,5.0,Euro 4,Full
2,2,Hyundai i30,949,137000,2011,NaN,Petrol,Hatchback,1.4L,Manual,5.0,5.0,Euro 5,NaN


In [3]:
data.shape

(3685, 14)

In [4]:
data.isnull().sum()

Unnamed: 0              0
title                   0
Price                   0
Mileage(miles)          0
Registration_Year       0
Previous Owners      1409
Fuel type               0
Body type               0
Engine                 45
Gearbox                 0
Doors                  25
Seats                  35
Emission Class         87
Service history      3145
dtype: int64

In [5]:
data.dtypes

Unnamed: 0             int64
title                 object
Price                  int64
Mileage(miles)         int64
Registration_Year      int64
Previous Owners      float64
Fuel type             object
Body type             object
Engine                object
Gearbox               object
Doors                float64
Seats                float64
Emission Class        object
Service history       object
dtype: object

In [6]:
# Taking out only the data I think I will need
df = data[['Price','Mileage(miles)','Registration_Year','Fuel type','Body type','Engine','Emission Class']]
df.shape

(3685, 7)

In [7]:
df.dropna()
df.rename(columns={'Mileage(miles)':'Mileage'}, inplace = True)

C:\Users\manua\AppData\Local\Temp\ipykernel_23460\2258436435.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns={'Mileage(miles)':'Mileage'}, inplace = True)


In [8]:
df.isnull().sum()

Price                 0
Mileage               0
Registration_Year     0
Fuel type             0
Body type             0
Engine               45
Emission Class       87
dtype: int64

In [9]:
df.head()

,Price,Mileage,Registration_Year,Fuel type,Body type,Engine,Emission Class
0,6900,70189,2016,Diesel,Hatchback,1.4L,Euro 6
1,1495,88585,2008,Petrol,Hatchback,1.2L,Euro 4
2,949,137000,2011,Petrol,Hatchback,1.4L,Euro 5
3,2395,96731,2010,Petrol,Hatchback,1.4L,Euro 4
4,1000,85000,2013,Diesel,Hatchback,1.3L,Euro 5


In [10]:
#Convert engine and emmission class coumns from objects to floats to make them useable
df['Engine'] = df['Engine'].str.replace('L','').astype(float)
df['Emission Class'] = df['Emission Class'].str.replace('Euro','').astype(float)
df.head()

C:\Users\manua\AppData\Local\Temp\ipykernel_23460\427814194.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Engine'] = df['Engine'].str.replace('L','').astype(float)
C:\Users\manua\AppData\Local\Temp\ipykernel_23460\427814194.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Emission Class'] = df['Emission Class'].str.replace('Euro','').astype(float)


,Price,Mileage,Registration_Year,Fuel type,Body type,Engine,Emission Class
0,6900,70189,2016,Diesel,Hatchback,1.4,6.0
1,1495,88585,2008,Petrol,Hatchback,1.2,4.0
2,949,137000,2011,Petrol,Hatchback,1.4,5.0
3,2395,96731,2010,Petrol,Hatchback,1.4,4.0
4,1000,85000,2013,Diesel,Hatchback,1.3,5.0


In [12]:
df['Body type'].value_counts()

Body type
Hatchback      2279
SUV             460
Saloon          368
Estate          171
MPV             153
Coupe           139
Convertible     109
Pickup            3
Combi Van         2
Minibus           1
Name: count, dtype: int64